In [52]:
rm(list=ls()) 

In [32]:
#install.packages("GA")

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [53]:
library(ggplot2)
library(dplyr)
library(GA)

In [54]:
departure_cell="ProB"
destination_cell="Mono"

In [55]:
imputed_data=read.table("Data/scimpute_count.csv", sep=",", header=TRUE, row.names=1)

In [56]:
rel_TFs=unlist(read.table("Data/TFS", header=FALSE))

In [57]:
rel_TFs[rel_TFs=="CEBPa"]="CEBPA"
rel_TFs[rel_TFs=="CEBPb"]="CEBPB"
rel_TFs[rel_TFs=="E2A"]="TCF3"
rel_TFs[rel_TFs=="EBF"]="EBF1"
rel_TFs[rel_TFs=="Eto2"]="CBFA2T3"
rel_TFs[rel_TFs=="Fli1"]="FLI1"
rel_TFs[rel_TFs=="Foxo1"]="FOXO1"
rel_TFs[rel_TFs=="Gata1"]="GATA1"
rel_TFs[rel_TFs=="Gata2"]="GATA2"
rel_TFs[rel_TFs=="Gfi1b"]="GFI1B"
rel_TFs[rel_TFs=="Ldb1"]="LDB1"
rel_TFs[rel_TFs=="Lmo2"]="LMO2"
rel_TFs[rel_TFs=="Lyl1"]="LYL1"
rel_TFs[rel_TFs=="Meis1"]="MEIS1"
rel_TFs[rel_TFs=="Mtgr1"]="CBFA2T2"
rel_TFs[rel_TFs=="Oct2"]="POU2F2"
rel_TFs[rel_TFs=="p300"]="EP300"
rel_TFs[rel_TFs=="P65"]="RELA"
rel_TFs[rel_TFs=="Pparg"]="PPARG"
rel_TFs[rel_TFs=="PU1"]="SPI1"
rel_TFs[rel_TFs=="Runx1"]="RUNX1"
rel_TFs[rel_TFs=="SCL"]="TAL1"
rel_TFs[rel_TFs=="Stat3"]="STAT3"
rel_TFs[rel_TFs=="Stat4"]="STAT4"
rel_TFs[rel_TFs=="Stat5a"]="STAT5A"
rel_TFs[rel_TFs=="Stat5b"]="STAT5B"
rel_TFs[rel_TFs=="Stat6"]="STAT6"

In [58]:
labels=unlist(read.table("Data/cell_types.txt"))

In [59]:
departure_ind=which(labels==departure_cell)
destination_ind=which(labels==destination_cell)

In [60]:
TF_data=imputed_data[rel_TFs,]
write.table(rel_TFs, file="TF_list.csv", col.names=FALSE, row.names=FALSE)

In [61]:
departure_data=TF_data[,departure_ind]
destination_data=TF_data[,destination_ind]
labels=c(rep.int(departure_cell, ncol(departure_data)),rep.int(destination_cell, ncol(destination_data)))

In [62]:
all_tf_data=cbind(departure_data, destination_data)
bin_tf_data=(all_tf_data>0)+0

In [63]:
bin_departure_data=t((departure_data>0)+0)
bin_destination_data=t((destination_data>0)+0)


In [65]:
departure_counts=table(apply(bin_departure_data,1, function(x){paste(x, collapse = "")}))
destination_counts=table(apply(bin_destination_data,1, function(x){paste(x, collapse = "")}))

In [66]:
n_cell_pairs=500

In [67]:
departure_inds=sample(1:length(departure_counts), size= n_cell_pairs, prob=departure_counts, replace=TRUE)
destination_inds=sample(1:length(destination_counts), size= n_cell_pairs, prob=destination_counts, replace=TRUE)

In [72]:
# SAMPLED NETWORK
parent_sampled_folder="Sampled_Network_Attractors"
dir.create(parent_sampled_folder)
num_sub_bn_sets=500
num_sel_bits=10
num_bits=30
n_genes=30

In [73]:
j=0;
for (j in 1:length(departure_inds)){       
        sub_folder=paste(parent_sampled_folder, j, sep="/")
        dir.create(sub_folder) 
        inds=matrix(0, nrow=num_sub_bn_sets, ncol=num_sel_bits)
        i=1
        ats=rbind(bin_departure_data[departure_inds[j],], bin_destination_data[destination_inds[j],])

        while (i<=num_sub_bn_sets){
            sampled=unlist(sample(1:num_bits,num_sel_bits))
            zz=apply(ats,1,function(x){binary2decimal(x[sampled])}) 
            #ats=sapply(attractors[as,], function(x){decimal2binary(x, n_genes)})
            if(!(any(table(zz)>1))){
                write.table(file=paste(sub_folder , "/attractors" , i,  ".txt", sep=""), row.names=FALSE, col.names=FALSE, t(zz))
                inds[i,]=sampled
                i=i+1
            }
        }
        for (node in 1:n_genes){
            aa=which(apply(inds,1,function(x){length(which(x==node))>0})==TRUE)
            if(length(unique(as.vector(inds[aa,])))<n_genes)
               print(paste("Node", node,   " is not seen together at least once with each of other nodes", sep=""))
        }
        write.table(file=paste(sub_folder , "/attractor_info.txt", sep=""), row.names=FALSE, col.names=FALSE, c(1, length(zz), length(zz)))
        write.table(file=paste(sub_folder ,"/ind_matrix.txt", sep=""), row.names=FALSE, col.names=FALSE, inds)
}

In [79]:
write.table(bin_departure_data[departure_inds,],quote=FALSE, file="departure_atts.csv", sep=",")

In [80]:
write.table(bin_destination_data[destination_inds,],quote=FALSE, file="destination_atts.csv", sep=",")

In [ ]:
departure_attractors=apply(bin_departure_data[departure_inds,], 1, binary2decimal)
destination_attractors=apply(bin_destination_data[departure_inds,], 1, binary2decimal)
dep_des_atts=cbind(departure_attractors, destination_attractors)
write.table(file="Attractors.csv", dep_des_atts, sep=, row.names=FALSE, col.names=FALSE)